# Text-to-SQL for Internal Data (with validation)

Goal: convert natural language questions into **safe, executable SQL** against a simple database.

What you’ll practice:
- Providing schema context
- Structured outputs for SQL
- Guardrails: allow only SELECT
- Executing and iterating on errors


## 1. Setup and Installation

**Estimated time:** ~60–90 minutes (with exercises)

### Install
If needed, install dependencies:
```bash
pip install -U openai pydantic pandas numpy scikit-learn
```

### Environment
Set your API key:
```bash
export OPENAI_API_KEY="..."
```

> **Note:** All example data in this notebook is synthetic (safe to share in training).

In [ ]:
import os

assert os.getenv('OPENAI_API_KEY'), "Set OPENAI_API_KEY in your environment"

## 2. Imports + API client

In [ ]:
from openai import OpenAI

client = OpenAI()  # uses OPENAI_API_KEY from env

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional, List
import sqlite3
import pandas as pd
import re
import json


## 3. Create a toy NYPL-style SQLite database


In [ ]:
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

cur.executescript("""
CREATE TABLE locations(
  location_id INTEGER PRIMARY KEY,
  borough TEXT,
  name TEXT
);

CREATE TABLE programs(
  program_id INTEGER PRIMARY KEY,
  title TEXT,
  location_id INTEGER,
  date TEXT,
  FOREIGN KEY(location_id) REFERENCES locations(location_id)
);

CREATE TABLE attendance(
  program_id INTEGER,
  attendees INTEGER,
  FOREIGN KEY(program_id) REFERENCES programs(program_id)
);

INSERT INTO locations VALUES
(1,'Manhattan','Mid-Manhattan'),(2,'Bronx','Bronx Library Center'),(3,'Brooklyn','Central Library');

INSERT INTO programs VALUES
(10,'Job Search Help',1,'2025-10-10'),
(11,'Kids Story Time',3,'2025-10-12'),
(12,'Digital Literacy 101',2,'2025-10-15'),
(13,'Job Search Help',2,'2025-11-02');

INSERT INTO attendance VALUES
(10,35),(11,52),(12,18),(13,28);
""")

conn.commit()

pd.read_sql_query("SELECT * FROM programs", conn)

## 4. Provide schema context to the model


In [ ]:
schema = """Database schema:
locations(location_id INTEGER PK, borough TEXT, name TEXT)
programs(program_id INTEGER PK, title TEXT, location_id INTEGER FK->locations.location_id, date TEXT 'YYYY-MM-DD')
attendance(program_id INTEGER FK->programs.program_id, attendees INTEGER)

Notes:
- Use JOINs between programs and locations via location_id
- attendance has one row per program_id
- Only write SELECT queries (read-only)
"""

## 5. Define a structured SQL output schema


In [ ]:
class SQLQuery(BaseModel):
    sql: str = Field(..., description="A single SQL SELECT query")
    explanation: str = Field(..., description="1-2 sentences explaining the query")
    safety_check: str = Field(..., description="State why this is read-only and safe")

SELECT_ONLY = re.compile(r"^\s*select\b", re.IGNORECASE)

def validate_select_only(sql: str):
    if not SELECT_ONLY.match(sql.strip()):
        raise ValueError("Only SELECT queries are allowed.")
    # crude extra guardrail:
    if re.search(r"\b(drop|delete|update|insert|alter)\b", sql, re.IGNORECASE):
        raise ValueError("Mutation keywords detected.")
    return sql

## 6. Generate SQL from a question


In [ ]:
SYSTEM = """You are a data assistant. Produce a SINGLE read-only SQL query.
Use only the provided schema. Do not guess tables/columns not listed.
Return structured output."""

def nl_to_sql(question: str) -> SQLQuery:
    resp = client.responses.parse(
        model="gpt-4o-2024-08-06",
        input=[
            {"role":"system","content": SYSTEM},
            {"role":"user","content": schema + "\nQuestion: " + question}
        ],
        text_format=SQLQuery
    )
    out = resp.output_parsed
    validate_select_only(out.sql)
    return out

q = nl_to_sql("How many total attendees were there for programs in the Bronx?")
q

## 7. Execute the query and view results


In [ ]:
df_res = pd.read_sql_query(q.sql, conn)
df_res

## 8. Iteration pattern: handle SQL errors safely

If execution fails (e.g., bad column), you can feed the error back with the schema and retry.


In [ ]:
def nl_to_sql_with_retry(question: str, max_tries: int = 2) -> SQLQuery:
    last_err=None
    for _ in range(max_tries):
        try:
            q = nl_to_sql(question)
            # dry run
            pd.read_sql_query(q.sql, conn).head()
            return q
        except Exception as e:
            last_err=e
            question = question + f"\n(If previous SQL failed, fix it. Error: {e})"
    raise last_err

nl_to_sql_with_retry("Show attendees by borough, highest first.")

## 9. Exercises


In [ ]:
# EXERCISE
# Write a question that requires a JOIN across programs, locations, and attendance (e.g., attendance by program title). Generate SQL and run it.

# TODO: your question here
raise NotImplementedError("TODO")


In [ ]:
# EXERCISE
# Strengthen validation: forbid multiple statements (e.g., ';' followed by another keyword).

def validate_select_only_strict(sql: str):
    # TODO
    raise NotImplementedError("TODO")

raise NotImplementedError("TODO")


In [ ]:
# EXERCISE
# Create a tiny 'eval set' of 3 questions and expected outputs (as numbers). Run them and check correctness.

# TODO: create questions + expected answers, then loop
raise NotImplementedError("TODO")
